In [42]:
%load_ext autoreload
%autoreload

In [43]:
import sys
sys.path.append('..')

import os
import json

import pandas as pd

In [44]:
datasets_path = os.path.join('..', 'datasets')

In [45]:
# Big color dataset
dpath_big = os.path.join(datasets_path, 'color-dict-big.json')
with open(dpath_big, 'r') as x:
    big = json.load(x)

In [46]:
# Standardize big color dataset
big_colors = pd.io.json.json_normalize(list(big['colors']))
big_colors = big_colors.drop(columns=['luminance'])
big_colors = big_colors.rename(columns={'rgb.r': 'r', 'rgb.g': 'g', 'rgb.b': 'b'})

In [48]:
# Set base colors
# Use the first 17 standard CSS1 colours
dpath_base = os.path.join(datasets_path, 'html-colors.json')
with open(dpath_base, 'r') as hc:
    base_col = json.load(hc)

# L1 distance between two hex colors (not the best way)
# perhaps need to convert it to Lab* space before calculating similarity
def min_distance_colorname(x):
    min_dist = 3000000
    min_color = 'black'
    x_int = int(x[1:], 16)
    for key, val in base_col.items():
        dist = abs(int(val[1:], 16) - x_int)
        if dist < min_dist:
            min_dist = dist
            min_color = key
            
    return min_color

# Try to get base color name based on default list
big_colors['basename'] = big_colors['hex'].apply(min_distance_colorname)
print(big_colors[13330:13338])

           hex               name    b    g    r basename
13330  #5d1f1e          Red Oxide   30   31   93   maroon
13331  #bb0044  Red Paracentrotus   68    0  187   silver
13332  #7b3539           Red Pear   57   53  123   maroon
13333  #dd0000        Red Pegasus    0    0  221   silver
13334  #f6b894        Red Perfume  148  184  246      red
13335  #ed1c24        Red Pigment   36   28  237      red
13336  #fa2a55           Red Pink   85   42  250      red
13337  #7c2946           Red Plum   70   41  124   maroon


In [79]:
# Small color dataset
dpath_small = os.path.join(datasets_path, 'color-dict-small.json')
with open(dpath_small, 'r') as x:
    small = json.load(x)

In [103]:
# Standardize small color dataset
small_colors = []
for colorPage in small['colorPage']:
    for entry in colorPage['colorEntry']:
        small_colors.append(entry)
small_colors = pd.io.json.json_normalize(list(small_colors))
small_colors = small_colors.rename(columns={
    'colorName': 'name', 'RGB8.red': 'r', 
    'RGB8.green': 'g', 'RGB8.blue': 'b'})
small_colors['name'] = small_colors['name'].apply(lambda n: re.sub('\(SW.*\)', '', n).strip())